In [221]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tqdm
import os
import csv
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
from tensorflow.python.framework import ops


In [222]:
image_size = 224

In [223]:
training_data_autistic_file_path = 'train/train/autistic'
training_data_not_autistic_file_path = 'train/train/non_autistic'
testing_data_file_path = 'test'
def create_training_and_validation_data(training_data_count,validation_data_count):
    all_autistic_data= []
    all_non_autistic_data=[]
    training_data = []
    validation_data = []
    for img in tqdm(os.listdir(training_data_autistic_file_path)):
        path = os.path.join(training_data_autistic_file_path, img)
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        all_autistic_data.append([np.array(image)/255.0,1])
    
    for img in tqdm(os.listdir(training_data_not_autistic_file_path)):
        path = os.path.join(training_data_not_autistic_file_path, img)
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        all_non_autistic_data.append([np.array(image)/255.0,0])

    full_data = all_autistic_data + all_non_autistic_data
    print(len(full_data))
    shuffle(full_data)

    for i in range(training_data_count):
        training_data.append(full_data[i])
    for i in range(training_data_count,training_data_count+validation_data_count):
        validation_data.append(full_data[i])

    shuffle(training_data)
    shuffle(validation_data)
    return training_data,validation_data
    

In [224]:
training_data,validation_data = create_training_and_validation_data(2056,480)

100%|██████████| 1268/1268 [00:02<00:00, 509.22it/s]2536



In [225]:
training_features = np.array([i[0] for i in training_data]).reshape(-1,image_size,image_size,3)
training_labels= [i[1] for i in training_data]
training_labels = keras.utils.to_categorical(training_labels, 2)

In [226]:
validation_features = np.array([i[0] for i in validation_data]).reshape(-1,image_size,image_size,3)
validation_labels= [i[1] for i in validation_data]
validation_labels = keras.utils.to_categorical(validation_labels, 2)

In [234]:
model = keras.Sequential(
    [
        keras.Input(shape=(224,224,3)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(2, activation="sigmoid"),
    ]
)

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 220, 220, 32)      2432      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 110, 110, 32)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 110, 110, 32)      128       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 106, 106, 64)      51264     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 53, 53, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 179776)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 179776)          

In [235]:
batch_size = 256
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(training_features, training_labels, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 1850 samples, validate on 206 samples
Epoch 1/15


In [ ]:
score = model.evaluate(validation_features, validation_labels, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])